In [1]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

La sintaxis del comando no es correcta.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1330k  100 1330k    0     0  1893k      0 --:--:-- --:--:-- --:--:-- 1903k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1253k  100 1253k    0     0  2464k      0 --:--:-- --:--:-- --:--:-- 2487k


In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/Pacolaz/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as Pacolaz

Initialized MLflow to track repo "Pacolaz/nyc-taxi-time-prediction"

Repository Pacolaz/nyc-taxi-time-prediction initialized!

https://dagshub.com/Pacolaz/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/95f7e9b740a14573ae117bfbbbc17c06', creation_time=1730258370273, experiment_id='0', last_update_time=1730258370273, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [10]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("Pacolaz/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

# Download file
# s3.download_file(
#     Bucket="nyc-taxi-time-prediction",  # name of the repo
#     Key="train_data.parquet",  #  remote path from where to download the file
#     Filename="local.csv",  # local path where to download the file
# )

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [11]:
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

# Download file
# s3.download_file(
#     Bucket="nyc-taxi-time-prediction",  # name of the repo
#     Key="train_data.parquet",  #  remote path from where to download the file
#     Filename="local.csv",  # local path where to download the file
# )

In [12]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [14]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [15]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:7.24715                           
[1]	validation-rmse:6.25749                           
[2]	validation-rmse:5.76748                           
[3]	validation-rmse:5.53527                           
[4]	validation-rmse:5.42608                           
[5]	validation-rmse:5.37003                           
[6]	validation-rmse:5.34328                           
[7]	validation-rmse:5.32848                           
[8]	validation-rmse:5.31684                           
[9]	validation-rmse:5.31045                           
[10]	validation-rmse:5.30648                          
[11]	validation-rmse:5.30050                          
[12]	validation-rmse:5.29764                          
[13]	validation-rmse:5.29207                          
[14]	validation-rmse:5.28516                          
[15]	validation-rmse:5.28406                          
[16]	validation-rmse:5.28025                          
[17]	validation-rmse:5.27933                          
[18]	valid

2024/10/29 21:25:49 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/10/29 21:25:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:25:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."

2024/10/29 21:26:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:26:33] WARNING: C:\buildkite-

[0]	validation-rmse:8.77829                                                     
[1]	validation-rmse:8.47186                                                     
[2]	validation-rmse:8.19051                                                     
[3]	validation-rmse:7.93269                                                     
[4]	validation-rmse:7.69267                                                     
[5]	validation-rmse:7.47659                                                     
[6]	validation-rmse:7.27882                                                     
[7]	validation-rmse:7.09135                                                     
[8]	validation-rmse:6.92871                                                     
[9]	validation-rmse:6.77631                                                     
[10]	validation-rmse:6.63723                                                    
[11]	validation-rmse:6.50936                                                    
[12]	validation-rmse:6.39611

2024/10/29 21:30:38 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/10/29 21:30:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:30:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."

2024/10/29 21:30:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:31:27] WARNING: C:\buildkite-

[0]	validation-rmse:6.19866                                                     
[1]	validation-rmse:5.55913                                                     
[2]	validation-rmse:5.42291                                                     
[3]	validation-rmse:5.38649                                                     
[4]	validation-rmse:5.37017                                                     
[5]	validation-rmse:5.35716                                                     
[6]	validation-rmse:5.34393                                                     
[7]	validation-rmse:5.34207                                                     
[8]	validation-rmse:5.33776                                                     
[9]	validation-rmse:5.33507                                                     
[10]	validation-rmse:5.33340                                                    
[11]	validation-rmse:5.33197                                                    
[12]	validation-rmse:5.32656

...

2024/10/29 21:41:58 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/10/29 21:41:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:41:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."

2024/10/29 21:42:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:42:10] WARNING: C:\buildkite-

[0]	validation-rmse:8.27181                                                    
[1]	validation-rmse:7.60194                                                    
[2]	validation-rmse:7.07362                                                    
[3]	validation-rmse:6.66019                                                    
[4]	validation-rmse:6.34204                                                    
[5]	validation-rmse:6.09920                                                    
[6]	validation-rmse:5.91372                                                    
[7]	validation-rmse:5.77337                                                    
[8]	validation-rmse:5.66757                                                    
[9]	validation-rmse:5.58684                                                    
[10]	validation-rmse:5.52638                                                   
[11]	validation-rmse:5.48091                                                   
[12]	validation-rmse:5.44649            

2024/10/29 21:43:13 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/10/29 21:43:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:43:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."

2024/10/29 21:43:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:43:21] WARNING: C:\buildkite-

100%|██████████| 10/10 [19:36<00:00, 117.67s/trial, best loss: 5.184572086147766]
[0]	validation-rmse:6.05532
[1]	validation-rmse:5.40037
[2]	validation-rmse:5.26460
[3]	validation-rmse:5.22876
[4]	validation-rmse:5.21300
[5]	validation-rmse:5.20725
[6]	validation-rmse:5.20211
[7]	validation-rmse:5.19922
[8]	validation-rmse:5.19808
[9]	validation-rmse:5.19656
[10]	validation-rmse:5.19826
[11]	validation-rmse:5.19786
[12]	validation-rmse:5.19705
[13]	validation-rmse:5.19588
[14]	validation-rmse:5.19482
[15]	validation-rmse:5.19392
[16]	validation-rmse:5.18929
[17]	validation-rmse:5.18471
[18]	validation-rmse:5.18373
[19]	validation-rmse:5.18284
[20]	validation-rmse:5.18370
[21]	validation-rmse:5.18353
[22]	validation-rmse:5.17709
[23]	validation-rmse:5.17698
[24]	validation-rmse:5.17828
[25]	validation-rmse:5.17898
[26]	validation-rmse:5.17806
[27]	validation-rmse:5.17552
[28]	validation-rmse:5.17266
[29]	validation-rmse:5.17418
[30]	validation-rmse:5.17489
[31]	validation-rmse:5.17432


2024/10/29 21:43:50 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/10/29 21:43:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\pacol\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:43:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/10/29 21:43:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/29 21:43:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimizatio

In [16]:
best_params

{'learning_rate': 0.6087681395216564,
 'max_depth': 47,
 'min_child_weight': 2.1228251865839693,
 'reg_alpha': 0.010706662118787161,
 'reg_lambda': 0.19200106429052627,
 'seed': 42,
 'objective': 'reg:squarederror'}

In [17]:
run_id = "15ad68e1a5fb493294e1444770c3eaa0"
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Successfully registered model 'nyc-taxi-model'.


MlflowException: API request to endpoint /api/2.0/mlflow/runs/get failed with error code 400 != 200. Response body: '"repo not associated with run"'

In [ ]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)